In [1]:
import numpy as np
import random
from scipy.stats import truncnorm
from scipy.optimize import check_grad
import itertools as it

In [2]:
if __name__ == "__main__":
    # Load data
    if ('train_images' not in globals()):  
        # In ipython, use "run -i homework2_template.py" to avoid re-loading of data
        train_images = np.load("mnist_train_images.npy")
        train_labels = np.load("mnist_train_labels.npy")
        validation_images = np.load("mnist_validation_images.npy")
        validation_labels = np.load("mnist_validation_labels.npy")
        test_images = np.load("mnist_test_images.npy")
        test_labels = np.load("mnist_test_labels.npy")

In [3]:
def softmax(row):
    """softmax function
    read in one line predicted result(np.array) return softmaxed predicted result(np.array)
    """
    row_sum = np.sum(np.exp(row), axis=1, keepdims=True)
    return np.array(1.0 * np.exp(row) / row_sum)

In [4]:
def relu(z1):
    """relu input np.array retuen np.array"""
    return np.maximum(z1, 0, z1)

In [5]:
def J(true_labels, y_hat):
    """true_labels and y_hat both in (1, 10) shape, true_labels should after one-hot"""
    return - np.mean(np.sum(np.array(true_labels) * np.log(np.array(y_hat) + 1e-5), axis=1))

In [15]:
class Network(object):
    
    def __init__(self, params):
        """hidden_layers
        
        Args : 
            params : {
                "hidden_layers" : [30, 40, 50], 
                "lr" : [0.001, 0.005, 0.01, 0.05, 0.1, 0.5],
                "batch_size" : [16, 32, 64, 128, 256],
                "epochs" : not this time
                "reg" : not this time}
        """
        self.fir = 784
        self.h1 = params["hidden_layers"]
        self.h2 = 10
            
        self.w1 = truncnorm(-1.0/np.sqrt(self.fir), 1.0/np.sqrt(self.fir), loc = 0, scale = 1).\
            rvs(self.fir * self.h1).reshape(self.fir, self.h1)
        self.b1 = np.expand_dims(np.ones(self.h1), axis = 0)
        
#         self.w1 = 0.01 * np.random.randn(self.fir, self.h1)
#         self.b1 = np.zeros((1, self.h1))
        
#         self.w2 = truncnorm(-1.0/np.sqrt(self.h1),1.0/np.sqrt(self.h1), loc = 0, scale = 1).\
#             rvs(self.h1 * self.h2).reshape(self.h1, self.h2)
#         self.b2 = np.expand_dims(np.ones(self.h2), axis = 0)
        
        self.w2 = 0.01 * np.random.randn(self.h1,self.h2)
        self.b2 = np.zeros((1,self.h2))

        self.lr = params["lr"]
        self.batch_size = params["batch_size"]
        self.epoches = 20
        self.reg = 0
    
    def feed_forward(self, single_train):
        
        z1 = single_train.dot(self.w1) + self.b1
        h1 = relu(z1)
        z2 = h1.dot(self.w2) + self.b2
        y_hat = softmax(z2)
        
        return y_hat
    
    def sgd(self, training, validation = None):
        """train with mini-batch stochastic gradient descent
        
        Args:
            training : (training_features, training_labels)
            validation : if validation is not None, caculate score for validation data
        """        
        n_train = len(training)
        for epoch in xrange(self.epoches):
            random.shuffle(training)
            mini_batches = [
                training[k: k + self.batch_size] for k in xrange(0, n_train, self.batch_size)]
            
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch)
        
            if validation != None :
                cost, accu = self.evaluate(validation)
                print "Epoch {0} : cost : {1}, accuracy : {2}".format(epoch, cost, accu)
            
    def update_mini_batch(self, mini_batch):
        """update the gradient descent"""
        
        origin_list = [self.w1, self.b1, self.w2, self.b2]
        nabla_list = [np.zeros_like(ele) for ele in origin_list]

        for x, y in mini_batch:
            dw1, db1, dw2, db2 = self.back_prop(x, y)
            for i, d_ele in enumerate([dw1, db1, dw2, db2]):
                nabla_list[i] += d_ele

        self.w1 -= (self.lr) * nabla_list[0]
        self.b1 -= (self.lr) * nabla_list[1]
        self.w2 -= (self.lr) * nabla_list[2]
        self.b2 -= (self.lr) * nabla_list[3]
    
    def back_prop(self, x, y):
        """returns dw1, db1, dw2, db2"""
        
        x = np.expand_dims(x, axis=0)
        y = np.expand_dims(y, axis=0)
        
        # feed forward process, store info in each layer
        z1 = x.dot(self.w1) + self.b1
        h1 = relu(z1)
        z2 = h1.dot(self.w2) + self.b2
        y_hat = softmax(z2)
        
        # gradient descent for w2
        dj_dw2 = (h1.T).dot(y_hat - y)

        # gradient descent for b2
        dj_db2 = (y_hat - y)

        # gradient descent for w1
        dj_dh1 = (y_hat - y).dot((self.w2).T)
        dj_dz1 = dj_dh1
        dj_dz1[h1 <= 0] = 0
        dj_dw1 = (x.T).dot(dj_dz1)

        # gradient descent for b1
        dj_db1 = dj_dz1
        
        return dj_dw1, dj_db1, dj_dw2, dj_db2

    def evaluate(self, validation_data):
        """return cost function and accruacy of test data"""
        x, y = zip(*validation_data)
        x = np.array(x) 
        f = np.array(y)
        
        pred_y = self.feed_forward(x)
        validation_cost = J(y, pred_y)
        
        validation_result = [(np.argmax(self.feed_forward(x)), np.argmax(y)) for (x, y) in validation_data]
        validation_accuracy = 1.0 * sum([int(x == y) for (x, y) in validation_result]) / len(validation_data)
        
        return validation_cost, validation_accuracy

In [16]:
params = {
    "hidden_layers" : 30, 
    "lr" : 1e-3,
    "batch_size" : 64}

network = Network(params)

network.sgd(zip(train_images, train_labels), zip(validation_images, validation_labels))

Epoch 0 : cost : 0.308377287118, accuracy : 0.9136
Epoch 1 : cost : 0.245541219914, accuracy : 0.9328
Epoch 2 : cost : 0.214253852991, accuracy : 0.938
Epoch 3 : cost : 0.18853502858, accuracy : 0.948
Epoch 4 : cost : 0.174031442265, accuracy : 0.9524
Epoch 5 : cost : 0.165388689087, accuracy : 0.9536
Epoch 6 : cost : 0.15024389401, accuracy : 0.9592
Epoch 7 : cost : 0.144458984468, accuracy : 0.9582
Epoch 8 : cost : 0.140685968856, accuracy : 0.9612
Epoch 9 : cost : 0.133892238529, accuracy : 0.9608
Epoch 10 : cost : 0.131011087433, accuracy : 0.9622
Epoch 11 : cost : 0.125808519769, accuracy : 0.9632
Epoch 12 : cost : 0.126024901887, accuracy : 0.963
Epoch 13 : cost : 0.120963819866, accuracy : 0.965
Epoch 14 : cost : 0.119806590523, accuracy : 0.9646
Epoch 15 : cost : 0.116638950322, accuracy : 0.9674
Epoch 16 : cost : 0.116894112478, accuracy : 0.9668
Epoch 17 : cost : 0.115245916975, accuracy : 0.9676
Epoch 18 : cost : 0.112089990017, accuracy : 0.9684
Epoch 19 : cost : 0.11799619

In [17]:
def find_best_hyperparameters(params_set, training, validation):
    
    params_names = sorted(params_set)
    combinations = [dict(zip(params_names, prod)) for prod in it.product(*(params_set[params_name] for params_name in params_names))]
    
    best_cost = 999
    best_param = {}
    
    for param in combinations:
        network = Network(param)
        network.sgd(training)
        validation_cost, _ = network.evaluate(validation)
        print param, validation_cost
        if validation_cost < best_cost:
            best_cost, best_param = validation_cost, param 
    
    return best_cost, best_param

In [18]:
params_set = {"hidden_layers" : [30, 40], "lr" : [ 0.01, 0.05], "batch_size" : [16, 32, 64]}
best_cost, best_param = find_best_hyperparameters(params_set, zip(train_images, train_labels), zip(validation_images, validation_labels))

{'lr': 0.01, 'hidden_layers': 30, 'batch_size': 16} 0.155601163059
{'lr': 0.05, 'hidden_layers': 30, 'batch_size': 16} 0.428617656988
{'lr': 0.01, 'hidden_layers': 40, 'batch_size': 16} 0.129053777354
{'lr': 0.05, 'hidden_layers': 40, 'batch_size': 16} 0.280194917033
{'lr': 0.01, 'hidden_layers': 30, 'batch_size': 32} 0.152826319933
{'lr': 0.05, 'hidden_layers': 30, 'batch_size': 32} 2.31428256441
{'lr': 0.01, 'hidden_layers': 40, 'batch_size': 32} 0.144604216982
{'lr': 0.05, 'hidden_layers': 40, 'batch_size': 32} 2.10697256953
{'lr': 0.01, 'hidden_layers': 30, 'batch_size': 64} 0.139669146717
{'lr': 0.05, 'hidden_layers': 30, 'batch_size': 64} 2.31112862687
{'lr': 0.01, 'hidden_layers': 40, 'batch_size': 64} 0.143870090566
{'lr': 0.05, 'hidden_layers': 40, 'batch_size': 64} 2.30988781137


In [19]:
print best_param

{'lr': 0.01, 'hidden_layers': 40, 'batch_size': 16}


In [20]:
network_test = Network(best_param)
network_test.sgd(zip(train_images, train_labels), zip(test_images, test_labels))

Epoch 0 : cost : 0.160174910338, accuracy : 0.9512
Epoch 1 : cost : 0.133626592051, accuracy : 0.9591
Epoch 2 : cost : 0.125676406288, accuracy : 0.963
Epoch 3 : cost : 0.118558162746, accuracy : 0.9652
Epoch 4 : cost : 0.113817139577, accuracy : 0.965
Epoch 5 : cost : 0.116495357458, accuracy : 0.9672
Epoch 6 : cost : 0.115578345192, accuracy : 0.9664
Epoch 7 : cost : 0.113126698501, accuracy : 0.9699
Epoch 8 : cost : 0.116774003229, accuracy : 0.968
Epoch 9 : cost : 0.10468873998, accuracy : 0.9703
Epoch 10 : cost : 0.116532868574, accuracy : 0.969
Epoch 11 : cost : 0.112363022409, accuracy : 0.9701
Epoch 12 : cost : 0.140069874108, accuracy : 0.9646
Epoch 13 : cost : 0.131174958875, accuracy : 0.967
Epoch 14 : cost : 0.133099556212, accuracy : 0.9672
Epoch 15 : cost : 0.145560494738, accuracy : 0.9622
Epoch 16 : cost : 0.139369965082, accuracy : 0.9666
Epoch 17 : cost : 0.135183335411, accuracy : 0.9693
Epoch 18 : cost : 0.137486369113, accuracy : 0.9676
Epoch 19 : cost : 0.13769358

### the check_grad

In [41]:
x = train_images[:1]
y = train_labels[:1]

network = Network(params)
w1, b1, w2 ,b2 = network.w1, network.b1, network.w2, network.b2

shape_list = []
for ele in [w1, b1, w2, b2]:
    shape_list.append(ele.shape)

def pack_wb(w1, b1, w2 ,b2) :
    packed_value = []
    for ele in [w1, b1, w2, b2]:
        packed_value += list(ele.flatten())

    return packed_value

packed_wb = pack_wb(w1, b1, w2 ,b2)

def unpack_wb(packed_wb, shape_list):
    unpacked_list = []
    index = 0
    old_index = 0
    for shape in shape_list:
        old_index = index
        shape_true = shape[0] * shape[1] 
        index += shape_true
        matrix = np.array(packed_wb[old_index : index]).reshape(shape)
        unpacked_list.append(matrix)

    return unpacked_list

def func_forward_w2(packed_wb, x, y, shape_list):
    
    w1, b1, w2, b2 = unpack_wb(packed_wb, shape_list)
    
    z1 = x.dot(w1) + b1
    h1 = relu(z1)
    z2 = h1.dot(w2) + b2
    y_hat = softmax(z2)
    
    return J(y, y_hat)

def func_back_w2(packed_wb, x, y, shape_list):
    
    w1, b1, w2, b2 = unpack_wb(packed_wb, shape_list)
    
    # feed forward process, store info in each layer
    z1 = x.dot(w1) + b1
    h1 = relu(z1)
    z2 = h1.dot(w2) + b2
    y_hat = softmax(z2)

    # gradient descent for w2
    dj_dw2 = (h1.T).dot(y_hat - y)

    # gradient descent for b2
    dj_db2 = (y_hat - y)

    # gradient descent for w1
    dj_dh1 = (y_hat - y).dot((w2).T)
    dj_dz1 = dj_dh1
    dj_dz1[z1 < 0] = 0
    dj_dw1 = (x.T).dot(dj_dz1)
    
    # gradient descent for b1
    dj_db1 = dj_dz1
    
    packed_der = pack_wb(dj_dw1, dj_db1, dj_dw2 ,dj_db2)
#     for ele in [dj_dw1, dj_db1, dj_dw2 ,dj_db2]:
#         print ele.shape

    return packed_der

# print check_grad(func_forward_w2, func_back_w2, packed_wb, x, y, shape_list)